In [ ]:
library(data.table)
library(bench)
library(purrr)
library(tidyverse)

files <- list.files("results", pattern = "\\.csv$", full.names = TRUE)
all_data <- lapply(files, function(file) {
    fread(file, data.table = FALSE) %>%
        mutate(tool = tools::file_path_sans_ext(basename(file)))
}) %>%
    bind_rows() %>%
    relocate(tool)

head(all_data)

,tool,dataset_size,min,median,mean,max,sd,ci95,mem_alloc,peak_mem_alloc,⋯,gc0_min,gc0_max,gc1_mean,gc1_median,gc1_min,gc1_max,gc2_mean,gc2_median,gc2_min,gc2_max
,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,dandelion,1000,345ms,353ms,361ms,426ms,23.1ms,16.5ms,4.71MB,1.08MB,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,dandelion,3000,930ms,1.01s,1.01s,1.06s,31.9ms,22.8ms,12.4MB,3.18MB,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,dandelion,6000,1.81s,1.85s,1.93s,2.36s,166ms,119ms,24.2MB,6.36MB,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,dandelion,10000,3.07s,3.12s,3.16s,3.33s,85.1ms,60.9ms,39.4MB,10.4MB,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,dandelion,30000,8.98s,9.02s,9.03s,9.11s,44.5ms,31.9ms,111MB,28.7MB,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,dandelion,60000,11.3s,11.7s,11.7s,12.2s,284ms,203ms,0B,0B,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [75]:
summarized <- all_data %>%
    dcast(tool ~ dataset_size, value.var="mean") #%>%
    #mutate(across(where(is.numeric), as.character))
summarized

Warning message in dcast(., tool ~ dataset_size, value.var = "mean"):
“The dcast generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is superseded and is no longer actively developed, and this redirection is now deprecated. Please do this redirection yourself like reshape2::dcast(.). In the next version, this warning will become an error.”


tool,1000,3000,6000,10000,30000,60000,100000,300000,600000,1000000
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
dandelion,361ms,1.01s,1.93s,3.16s,9.03s,11.7s,15.3s,28.5s,51.8s,1.37m
immunarch,1.23s,3.03s,6.06s,9.64s,27.4s,40.4s,1.18m,3.28m,4.11m,5.98m
scirpy,266ms,688ms,1.35s,2.24s,6.34s,13.2s,21.2s,1.16m,2.3m,3.79m
scRepertoire1,1.19s,2.08s,6.14s,9.46s,56.9s,5.58m,NA,NA,NA,NA
scRepertoire2,130ms,301ms,622ms,809ms,2.41s,5.06s,7.94s,24.8s,53s,1.35m


In [76]:
ci_df <- all_data %>%
    dcast(tool ~ dataset_size, value.var="ci95") %>%
    mutate(across(where(is.numeric), as.character))
ci_df

Warning message in dcast(., tool ~ dataset_size, value.var = "ci95"):
“The dcast generic in data.table has been passed a data.frame and will attempt to redirect to the relevant reshape2 method; please note that reshape2 is superseded and is no longer actively developed, and this redirection is now deprecated. Please do this redirection yourself like reshape2::dcast(.). In the next version, this warning will become an error.”


tool,1000,3000,6000,10000,30000,60000,100000,300000,600000,1000000
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
dandelion,16.5ms,22.8ms,119ms,60.9ms,31.9ms,203ms,764ms,478ms,1.36s,1.07s
immunarch,62.2ms,104.52ms,177.7ms,429.45ms,1.52s,1.54s,25.91s,1.94m,26.53s,4s
scirpy,37.4ms,13.3ms,43.9ms,40.8ms,96.9ms,563ms,170ms,675ms,1.69s,2.3s
scRepertoire1,718.22ms,74.17ms,937.88ms,1.72s,3.36s,1.63m,NA,NA,NA,NA
scRepertoire2,28.14ms,31.84ms,449.19ms,296.17ms,440.56ms,356.64ms,600.98ms,955.16ms,5.57s,2.46s


In [86]:
merged_df <- summarized
merged_df[-1] <- as.data.frame(mapply(function(a, b) paste0(a, "±", b), summarized[-1], ci_df[-1], SIMPLIFY = FALSE))
merged_df[merged_df == "NA±NA"] <- "-"
desired_order <- c("scRepertoire2", "dandelion", "scirpy", "immunarch", "scRepertoire1")
merged_df <- merged_df[match(desired_order, merged_df[[1]]), ]
fwrite(merged_df, "result_summary.csv")